完成了整个存储和获取，获取了相关的切分文档之后，现在我们需要将它们传递给语言模型，以获得答案。这个过程的一般流程如下：首先问题被提出，然后我们查找相关的文档，接着将这些切分文档和系统提示一起传递给语言模型，并获得答案

借鉴prompt与chain组件，我们组件第一个RAG应用

In [20]:
import os
# 加载embedding模型
from langchain.vectorstores import Chroma
from langchain.embeddings.ollama import OllamaEmbeddings
embeddings_model=OllamaEmbeddings(model='quentinz/bge-large-zh-v1.5:latest',base_url='http://192.168.3.155:11434')

persist_directory="VectorStore2" # 数据库保存路径
if os.path.exists(persist_directory):
    # 或者如果你想从头开始，可以使用以下代码来创建新的向量存储库，但记得先删除旧的持久化文件夹。
    db = Chroma(persist_directory=persist_directory,embedding_function=embeddings_model)
else:
    # step1：加载数据
    from langchain.document_loaders import DirectoryLoader
    from langchain.document_loaders import TextLoader
    loader = DirectoryLoader(
        "./documents/data/",
        glob="*.txt",
        loader_cls=TextLoader,
        loader_kwargs={"encoding":"utf-8"},
        show_progress=True)
    documents=loader.load()

    # step2：分块
    from langchain.text_splitter  import CharacterTextSplitter
    text_splitter = CharacterTextSplitter(chunk_size=512,chunk_overlap=32)
    split_documents=text_splitter.split_documents(documents)

    # step3：嵌入
    db = Chroma.from_documents(split_documents, embeddings_model, persist_directory=persist_directory)
    db.persist()

# step4：检索器
retriever = db.as_retriever()

In [21]:
# 处理检索器输出
def format_docs(docs):
    # print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

# 创建PromptTemplate
from langchain.prompts import PromptTemplate
template="""请严格按照下面的上下文内容回答问题，不要试图编造答案，如果你不知道答案，就回答不知道
总是在答案结束时说“谢谢你的提问！”
{context}
问题：{question}
"""
prompt = PromptTemplate(input_variables=["context", "question"],template=template,)

from langchain_ollama import OllamaLLM
# step0：初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
llm  = OllamaLLM(model=model_name)

# 创建检索-问答链
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
question = "风雪山神庙发生了什么？"
response = chain.invoke(question)
print(response)

在《水浒传》中，风雪山神庙这一情节讲述了林冲的遭遇及其转变的关键事件：

1. 林冲被高俅陷害，误入白虎堂后遭冤枉发配至沧州。

2. 在沧州，林冲被安排在柴大官人家做管营。但高俅等人依然对他步步紧逼，最后通过陆谦（原属林冲）密谋火烧草料场，企图将林冲置于死地。

3. 林冲在风雪交加的夜晚中发现草料场被烧毁，自己也险些丧命。他来到山神庙躲避风雪，并在此处听到门外陆谦、富安等人对他的陷害之事的对话。

4. 听完真相后，林冲怒火中烧，在山神庙前杀了陆谦等人。为了避祸，他选择离开沧州。

这一段情节不仅是林冲性格转变的重要转折点——从逆来顺受到奋起反抗；同时也是推动整个水浒故事进一步发展的关键事件之一，因为它为后续的梁山好汉聚集提供了契机。


以上过程是将RAG过程拆解为以下三步：

1. 通过检索器查找到相关文档，并提取相关文档的page_content作为llm回答的上下文
2. 将page_content追加到prompt中
3. 将prompt输入到llm中，然后使用StrOutputParser解析模型输出

这是通过自定义chain实现最原始的RAG，有很多可以改进的点，单就第一点，所有page_content合并到prompt就有问题，因为prompt的长度有限，所以无法合并大文档，以后通过langchian已经实现的chian：RetrievalQA解决这个问题

In [27]:
from langchain.chains import RetrievalQA
question = "风雪山神庙发生了什么？"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type="refine")

result = qa_chain({"query": question})
print(result["result"])

根据提供的新背景信息，可以将原本的答案进行稍微调整和补充：

在被奸臣高俅等人陷害后，林冲通过好友蔡京的帮助获得了一封推荐信前往梁山泊避难。在前往梁山的路上遇到了大雪天气。林冲一行人在一个酒店休息时，写下了诗作并悬挂于墙上，未曾想到另一位顾客朱贵（旱地忽律）就是梁山好汉之一，他通过诗作认出了林冲的身份。

随后，宋江带着随从燕顺等人也途径此地，并在此遇见了正在歇息的林冲一行人。二人相见后，宋江提议与他们结交并共同前往梁山泊。与此同时，酒店内另一位顾客吕方（小温侯）和郭盛（赛仁贵）也在争斗中偃旗息鼓，同意握手言和，并由秦明等人带来的人马作证。

具体事件经过如下：

- 林冲在蔡京的帮助下获得去往梁山的路线指引。
- 他们在大雪中行进，期间在酒店歇息并写下诗作。
- 酒店里另一位顾客朱贵通过林冲的诗歌认出了他，并与之对话，决定帮助林冲安全抵达梁山泊。
- 宋江带着燕顺等人在此地遇见了林冲一行人，提议共同前往梁山。
- 与此同时，酒店内吕方和郭盛也在争斗中偃旗息鼓，同意握手言和。

这样不仅保留了原先关键情节，还进一步补充了宋江一行人与吕方、郭盛之间的会面及和平解决争端的情节。


参数中新增chain_type，控制检索的文档如何输入到llm中，实际上有以下三种思路：

![alt text](使用langchain实现RAG过程-20241105161118.png)

|方法|步骤|优缺点|
|---|---|---|
|MapReduce|首先将每个独立的文档单独发送到语言模型以获取原始答案。然后，这些答案通过最终对语言模型的一次调用组合成最终的答案|涉及了更多对语言模型的调用，但它的优势在于可以处理任意数量的文档|
|Refine|对于每一个文档，会调用一次 LLM。但改进之处在于，最终输入语言模型的 Prompt 是一个序列，将之前的回复与新文档组合在一起，并请求得到改进后的响应|增强了上下文信息，从而解决信息分布在不同文档的问题|
|Map-Rerank|每个检索到的文档单独回答，并输出答案的评分，最后选择评分高的输出|要求文档分割准确，如果上下文分布在不同文档上，会造成答案缺失|
